In [153]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import Image
import os

if not os.path.exists("images"):
    os.mkdir("images")

In [154]:
generate_report = False
def plot(trace_num, x_data, y_data, xlable = 'xlable', ylable = 'ylable',
                legend = 'legend', title = 'title', mode='lines'):
    plot.counter += 1
    fig_name = 'images/' + str(plot.counter) + '.jpg'
    fig = go.Figure()
    for i in range(trace_num):
        fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode=mode, name=legend[i], showlegend = True))
    fig.update_layout(
        title=go.layout.Title(
            text=title,
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=xlable
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text=ylable
            )
        )
    )
    if generate_report is True:
        fig.write_image(fig_name)
        display(Image(fig_name))
    else:
        fig.show()
plot.counter = 0

## First part

In [155]:
def exp_smooth_forward(data, alpha, init=0):
    out = np.empty(data.size)
    out[0] = init
    for i in range(1, data.size):
        out[i] = out[i-1] + alpha*(data[i]-out[i-1])
    return out

In [156]:
def exp_smooth_backward(data, alpha, init=0):
    out = np.empty(data.size)
    out[-1] = init
    for i in reversed(range(data.size-1)):
        out[i] = out[i+1] + alpha*(data[i]-out[i+1])
    return out

In [200]:
def exp_smooth(data, alpha, init=0):
    return exp_smooth_backward(exp_smooth_forward(data, alpha, init), alpha, exp_smooth_forward(data, alpha, init)[-1])

In [157]:
def running_mean(data, M):
    out = pd.Series(data).rolling(window = M, center = True).mean()
    out[:int(M/2)+1] = out[int(M/2)]
    out[-int(M/2):] = out[data.size-int(M/2)-1]
    return out

In [158]:
def random_walk_tragectory(init, size, sigma):
    out = np.empty(size)
    out[0] = init
    w = np.random.normal(loc=0, scale=sigma, size=size)
    for i in range(1, size):
        out[i] = out[i-1]+w[i]
    return out

In [159]:
def measured_trajectory(data, sigma):
    return np.add(data, np.random.normal(loc=0, scale=sigma, size=data.size))

In [160]:
N = 300
sigma_w = 28
sigma_n = 97
num_points = np.linspace(1, N, num=N)
X = random_walk_tragectory(10, N, sigma_w)
z = measured_trajectory(X, sigma_n)

In [161]:
plot(2, [num_points, num_points], [X, z])

In [162]:
ksi = sigma_w**2/sigma_n**2
alpha = (-ksi + (ksi**2 + 4*ksi)**0.5)/2
print('Exponential smoothing parameter alpha = {:.2f}'.format(alpha))

Exponential smoothing parameter alpha = 0.25


In [163]:
exp_smooth_for = exp_smooth_forward(z, alpha, z[0])

In [164]:
exp_smooth_back = exp_smooth_backward(exp_smooth_for, alpha, exp_smooth_for[-1])

In [165]:
M = 7
run_mean = running_mean(z, M)

In [166]:
plot(5, [num_points, num_points, num_points, num_points, num_points], 
        [X, z, exp_smooth_for, exp_smooth_back, run_mean],
     legend=['true trajectory', 'measurements', 'forward exponential smoothing',
             'backward exponential smoothing', 'running mean'])

In [167]:
def deviation(X, z):
    return np.sum((X[:]-z[:])**2)

def variability(X):
    I = np.empty(X.size-2)
    for i in range(X.size-2):
        I[i] = (X[i+2] - 2*X[i+1] + X[i])**2
    return np.sum(I)

In [170]:
dev_rm = deviation(run_mean, z)
dev_es = deviation(exp_smooth_back, z)
var_rm = variability(run_mean)
var_es = variability(exp_smooth_back)
print(('deviation indicator: running mean                     = {:.2f}\n'+
      '                     backward exponential smoothing   = {:.2f}').format(dev_rm, dev_es))
print(('variability indicator: running mean                   = {:.2f}\n'+
      '                       backward exponential smoothing = {:.2f}').format(var_rm, var_es))


deviation indicator: running mean                     = 2760430.61
                     backward exponential smoothing   = 2649620.39
variability indicator: running mean                   = 232251.56
                       backward exponential smoothing = 18114.95


## Second part

### First trajectory

In [171]:
def motion_tragectory(size, T, sigma_a, X_init=0, V_init=0):
    out = np.empty(size)
    V = np.empty(size)
    out[0] = X_init
    V[0] = V_init
    a = np.random.normal(loc=0, scale=sigma_a, size=size)
    for i in range(1, size):
        V[i] = V[i-1] + a[i-1]*T
        out[i] = out[i-1] + V[i-1]*T + (a[i-1]*T**2)/2
    return out

In [172]:
N = 300
X = motion_tragectory(N, 0.1, np.sqrt(10), X_init=5, V_init=0)
z = measured_trajectory(X, np.sqrt(500))
num_points = np.linspace(1, N, num=N)

In [173]:
plot(2, [num_points, num_points], [X, z],
     legend=['true trajectory', 'measured trajectory'])

In [186]:
run_mean = running_mean(z, 1)
min_indicator = variability(run_mean)
optimal_M = 1
var_ind = np.empty(N)
dev_ind = np.empty(N)
for M in range (N):
    run_mean = running_mean(z, M)
    var_ind[M] = variability(run_mean)
    dev_ind[M] = deviation(run_mean, z)

In [188]:
plot(2, [num_points, num_points], [var_ind, dev_ind],
     legend=['variability','deviation'])

In [199]:
plot(3, [num_points, num_points, num_points], [X, z, running_mean(z, 50)],
     legend=['true trajectory', 'measured trajectory', 'running mean'])

In [205]:
exp_sm = exp_smooth(z, 0.1)
var_ind = np.empty(N)
dev_ind = np.empty(N)
for i, alpha in enumerate(np.linspace(0, 1, N)):
    exp_sm = exp_smooth(z, alpha)
    var_ind[i] = variability(exp_sm)
    dev_ind[i] = deviation(exp_sm, z)

In [207]:
plot(3, [num_points, num_points, num_points], [var_ind, dev_ind, np.linspace(0, 1, N)],
     legend=['variability','deviation', 'alpha'])

In [213]:
plot(3, [num_points, num_points, num_points], [X, z, exp_smooth(z, 0.06)],
     legend=['true trajectory', 'measured trajectory', 'exponential smoothing'])

### Second trajectory

In [216]:
def cyclic_trajectory(size, T, sigma, A_init=0):
    A = np.empty(size)
    out = np.empty(size)
    w = np.random.normal(loc=0, scale=sigma, size=size)
    W = 2*np.pi/T
    for i in range(1, size):
        A[i] = A[i-1] + w[i]
        out[i] = A[i] * np.sin(W*i+3)
    return out

In [219]:
N = 200
X = cyclic_trajectory(N, 32, 0.08, A_init=1)
z = measured_trajectory(X, 0.05)

In [222]:
plot(3, [num_points, num_points, num_points], [X, z, running_mean(z, 13)],
     legend=['true trajectory', 'measured trajectory', 'running mean M=13'])

In [243]:
M = 25
N = 200
T = 1
X = cyclic_trajectory(N, T, 0.08, A_init=1)
z = measured_trajectory(X, 0.05)
plot(3, [num_points, num_points, num_points], [X, z, running_mean(z, M)],
     legend=['true trajectory', 'measured trajectory', 'running mean M=25'])

**T =0-13, 25 - no oscillation**
**T = **